<a href="https://www.kaggle.com/code/keerthi4701/fake-news-classification-simple-model?scriptVersionId=138818183" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-and-real-news-dataset/True.csv
/kaggle/input/fake-and-real-news-dataset/Fake.csv


# Import Datasets

In [2]:
real = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")
fake = pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")

print("real: ",real.shape,"\nfake: ",fake.shape) ## datasets seem to be balanced. NO imbalance

real:  (21417, 4) 
fake:  (23481, 4)


In [3]:
display('real news',real.head())
display("fake news",fake.head())

'real news'

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


'fake news'

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
## EDA
print("real:\n",real.subject.value_counts(),"\n\nfake:\n",fake.subject.value_counts())

real:
 politicsNews    11272
worldnews       10145
Name: subject, dtype: int64 

fake:
 News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64


In [5]:
## sample news from datasets
print(real.title[0],"\t Real")
print(fake.title[0],"\t Fake")

As U.S. budget fight looms, Republicans flip their fiscal script 	 Real
 Donald Trump Sends Out Embarrassing New Year’s Eve Message; This is Disturbing 	 Fake


# Feature Engineering

In [6]:
## Label Encoding
real['label'] = 1
fake['label'] = 0

In [7]:
import spacy
nlp = spacy.load("en_core_web_lg")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [8]:
real['vector'] = real.title.apply(lambda x: nlp(x).vector)
fake['vector'] = fake.title.apply(lambda x: nlp(x).vector)

In [9]:
concat_data = pd.concat([real[['vector','label']],fake[['vector','label']]])
concat_data[:5]

,vector,label
0,"[-2.0529835, -0.569027, -3.1433282, 0.55970275...",1
1,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ...",1
2,"[0.557869, 2.2574153, 0.1913623, -0.041898414,...",1
3,"[-1.4035959, 0.07810003, -0.17810063, 2.512658...",1
4,"[0.010546171, 1.6784946, -0.5903046, 0.4184245...",1


# Model Building

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [11]:
X_train, X_test, y_train, y_test = train_test_split(concat_data.vector,
                                                    concat_data.label,
                                                    test_size = 0.2, 
                                                    random_state = 1,
                                                    stratify = concat_data.label)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(35918,) (35918,)
(8980,) (8980,)


In [12]:
X_train = np.stack(X_train)
X_test = np.stack(X_test)

X_train

array([[-0.23301356,  4.5374684 , -2.3352568 , ...,  1.769452  ,
        -3.347071  ,  3.5270379 ],
       [-0.31101125,  0.70727515, -3.050925  , ..., -1.6768737 ,
        -1.6511288 ,  1.9876038 ],
       [ 0.39467925,  1.669555  ,  0.59880286, ...,  1.7896794 ,
        -2.3822052 ,  1.9330199 ],
       ...,
       [-3.259486  ,  2.728493  , -3.6598694 , ..., -0.78519315,
        -2.2183661 ,  0.13245803],
       [-0.92145747, -0.53599155,  1.6962891 , ..., -0.33306703,
        -0.24070585, -0.43264183],
       [-1.3668569 ,  1.0633429 , -0.39573893, ..., -0.5022214 ,
        -1.0951021 ,  0.8527349 ]], dtype=float32)

In [13]:
Naive_bayes = make_pipeline(
    MinMaxScaler(),
    MultinomialNB()                        ## Naive_Bayes Classifier
    
)

DT = make_pipeline(
    MinMaxScaler(),
    DecisionTreeClassifier()              ## Decision_Tree_Classifier
    
)

KNN = make_pipeline(
    MinMaxScaler(),
    KNeighborsClassifier()               ## K_Nearest_Neighbours_Classifier
    
)

LogReg = make_pipeline(
    MinMaxScaler(),
    LogisticRegression(max_iter=1000)    ## Logistic_Regression_Classifier
    
)

SVM = make_pipeline(
    MinMaxScaler(),
    SVC()                               ## Support_Vector_Machines
    
)

In [14]:
def predict(model):
    model.fit(X_train,y_train)
    print(classification_report(y_test, model.predict(X_test)))

In [15]:
predict(SVM)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [16]:
predict(LogReg)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4696
           1       1.00      1.00      1.00      4284

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [17]:
predict(KNN)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4696
           1       1.00      0.99      0.99      4284

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [18]:
predict(DT)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      4696
           1       0.99      0.98      0.98      4284

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [19]:
predict(Naive_bayes)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4696
           1       0.98      0.98      0.98      4284

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



# # Thanks for viewing 👍